In [ ]:
#import all dependencies
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [ ]:
#download bert packages
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=f2e7f5d37d13219020dae0d53e5de435307e7f04a8561a3521e0d0f7efc19c21
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=5910fa0f9c1572a80f4bb68d04dbf96d95fd7afb4c107f908f847b1f0fead2d0
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=5b72e6f0e8589650737064f64e12e5a48615a4162730354212eae38a33dec27d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 4.5MB/s 


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Input
from keras.layers import Bidirectional, GlobalMaxPool1D, LSTM, GRU, SimpleRNN
from keras.initializers import Constant
from keras.models import Model
import matplotlib.pyplot as plt
import os
import sys

Using TensorFlow backend.


# loading files

In [ ]:
#get data from personal google drive folders
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/My Drive/twitterfakenews/train.csv", engine="python", encoding = "latin1")

In [140]:
test = pd.read_csv("/content/drive/My Drive/twitterfakenews/test.csv", engine="python", encoding = "latin1")

In [ ]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
data.drop(["keyword", "location", "id"], axis = 1, inplace = True)

In [ ]:
test.drop(["keyword", "location"], axis = 1, inplace = True)

In [ ]:
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


# Cleaning tweets

In [ ]:
#clean tweet
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) #remove mentions
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)
  tweet = tweet.lower()
  return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.target.values #get in numpy format

In [ ]:
test_clean = [clean_tweet(tweet) for tweet in test.text]

# Tokenization

In [ ]:
Fulltokenizer = bert.bert_tokenization.FullTokenizer
#bert layer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1", trainable = False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = Fulltokenizer(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("Samantha loves final fantasy")

['[UNK]', 'love', '##s', 'final', 'fantasy']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Samantha loves final fantasy"))

[100, 8451, 8118, 10591, 12436]

In [ ]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
data_inputs = [encode_sentence(sent) for sent in data_clean]

# Dataset creation

In [ ]:
#padding tokens per batch all the sentences do not need to have the same length
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
#random.shuffle(data_with_len) #not always necessary
data_with_len.sort(key = lambda x: x[2]) #sort based on length get access to the third element
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len] #remove short sentences

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, output_types=(tf.int32, tf.int32))

In [ ]:
all_test = tf.data.Dataset.from_generator(lambda: test_inputs, output_types=(tf.int32, tf.int32))

In [ ]:
next(iter(all_dataset))

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10353,  9558], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
NB_BATCHES =math.ceil(len(sorted_all)/BATCH_SIZE)

In [ ]:
NB_BATCHES

238

In [ ]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 4), dtype=int32, numpy=
 array([[10353,  9558,     0,     0],
        [ 8811, 10841,     0,     0],
        [ 8413, 10447,     0,     0],
        [12139,  8542,     0,     0],
        [10451,  8180,     0,     0],
        [ 9193,  8451,     0,     0],
        [ 9978,  8310, 11338,     0],
        [ 8792,  8902,  8798,     0],
        [ 8174,  9931,   106,     0],
        [10100,  8613,  9069,     0],
        [10100,  8613,  9069,     0],
        [ 9867, 13214,  8118,     0],
        [13158,  8197,   163,     0],
        [10951, 12670,  8303,     0],
        [ 8363, 11486,  8118,     0],
        [ 8363, 11486,  8118,     0],
        [10092,  8179, 11229,     0],
        [12289,  9917,  8863,     0],
        [12289,  9917,  8863,     0],
        [12289,  9917,  8863,     0],
        [12605,  8458,  8487,     0],
        [  151, 10235, 11229,     0],
        [ 8450,  9238, 11140,     0],
        [11515, 11714,  8221,     0],
        [11515, 11714,  8221,     0],
  

In [ ]:
NB_BATCHES_TEST = NB_BATCHES //10

In [ ]:
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Model Building

In [ ]:
class LSTM(tf.keras.Model):
  
  def __init__(self, vocab_size,
               emb_dim = 128,
               LSTM_units = 50,
               dropout_rate = 0.1,
               training = False,
               name = "lstm"):
    super(LSTM, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size, emb_dim)
    self.dense_1 = layers.Bidirectional(layers.LSTM(LSTM_units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
    self.pool = layers.GlobalAveragePooling1D()
    self.dropout = layers.Dropout(rate =dropout_rate)
    self.last_dense = layers.Dense(units =1, activation= "sigmoid")

  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.dense_1(x)
    x_2 = self.pool(x_1)
    x_3 = self.dropout(x_2)
    output = self.last_dense(x_3)

    return output


In [ ]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
LSTM_UNITS = 50
NB_CLASSES = 2
DROPOUT_RATE = 0.1
NB_EPOCHS = 10

In [ ]:
lstm = LSTM(vocab_size= VOCAB_SIZE,
            emb_dim = EMB_DIM,
            LSTM_units = LSTM_UNITS,
            dropout_rate = DROPOUT_RATE)

In [ ]:
Dcnn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])


# Results

In [ ]:
lstm.fit(train_dataset, epochs = NB_EPOCHS, validation_split = .1)

In [ ]:
def get_prediction(sent):
  tokens = encode_sentence(sent)
  inputs = tf.expand_dims(tokens, 0)

  output = Dcnn(inputs, training = False)

  target = math.floor(output*2)
  return output

In [123]:
range(len(preds))

range(0, 3263)

In [ ]:
preds = [get_prediction(sent) for sent in test_clean]

In [114]:
y_preds = []
for i in range(len(preds)): 
  y_hat = preds[i].numpy()
  y_preds.append(y_hat)

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9999869]], dtype=float32)>]

In [133]:
y_hat = np.stack(y_preds, axis=0)

In [137]:
y_pred = np.matrix(y_hat)

In [138]:
pred = pd.DataFrame(y_pred)

In [144]:
pred = pred.T

In [145]:
ids = pd.DataFrame(test[['id']])
df = pd.concat([ids, pred], axis = 1)
df.columns = ['id', 'target']
df.set_index('id', inplace=True)

In [146]:
df['target'] = df['target'].apply(lambda x: 0 if x < .5 else 1)

In [148]:
df.head

<bound method NDFrame.head of        target
id           
0           1
2           1
3           1
9           1
11          1
...       ...
10861       1
10865       1
10868       1
10874       1
10875       1

[3263 rows x 1 columns]>

In [147]:
#save
df.to_csv("/content/drive/My Drive/twitterfakenews/mysubmissio_bert.csv")

scores 0.74961 which is not bad